In [3]:
import pandas as pd
import numpy as np
from utils import read_file, tokenize_latex
df = read_file("data/1601/*")
df["processed"] = df["text"].apply(lambda x: tokenize_latex(x))

error with: data/1601/1601.06597.json


In [4]:
from gensim import corpora
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
equations = df["processed"].tolist()
dictionary = corpora.Dictionary(equations)

corpus = [dictionary.doc2bow(eq) for eq in equations]
corpora.MmCorpus.serialize('/tmp/equations.mm', corpus)
# our vector space model
print(corpus[1:5])

2017-11-04 18:38:51,876 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-11-04 18:38:52,751 : INFO : adding document #10000 to Dictionary(19919 unique tokens: [u'\\xi\\leq', u'4E+6F', u'0.000004', u'20\\%', u'i|S']...)
2017-11-04 18:38:53,360 : INFO : adding document #20000 to Dictionary(33619 unique tokens: [u'\\xi\\leq', u'eF', u'q-1', u'q-2', u'+i+1']...)
2017-11-04 18:38:54,097 : INFO : adding document #30000 to Dictionary(45883 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'JN2', u'q-1']...)
2017-11-04 18:38:54,908 : INFO : adding document #40000 to Dictionary(57458 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'JN2', u'q-1']...)
2017-11-04 18:38:55,629 : INFO : adding document #50000 to Dictionary(70025 unique tokens: [u'\\xi\\leq', u'eF', u'homomorphism', u'-5AB\\sqrt', u'JN2']...)
2017-11-04 18:38:56,006 : INFO : built Dictionary(76641 unique tokens: [u'\\xi\\leq', u'eF', u'homomorphism', u'-5AB\\sqrt', u'JN2']...) from 56379 documents (total 37

[[(2, 1), (4, 1), (5, 1), (6, 3), (8, 1), (11, 3), (14, 1), (15, 3), (16, 4), (17, 3), (21, 2), (23, 6), (25, 6), (26, 1), (27, 1), (28, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 2), (11, 3), (14, 1), (15, 2), (16, 3), (17, 5), (18, 1), (19, 4), (21, 2), (23, 7), (24, 1), (25, 7), (27, 2), (29, 1), (30, 1), (31, 1)], [(1, 1), (2, 1), (4, 1), (5, 1), (8, 1), (10, 1), (14, 1), (16, 2), (17, 1), (21, 2), (23, 8), (25, 8), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)], [(0, 1), (1, 1), (2, 1), (4, 3), (5, 3), (6, 1), (8, 1), (10, 1), (12, 3), (14, 1), (17, 2), (18, 2), (21, 2), (23, 8), (25, 8), (34, 1), (45, 1), (46, 1), (47, 2), (48, 2)]]


In [16]:
from gensim import corpora, models, similarities, matutils
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from gensim import similarities
index = similarities.Similarity("index/", corpus_tfidf, len(dictionary.keys()), num_best = 10)

2017-11-04 18:42:11,930 : INFO : collecting document frequencies
2017-11-04 18:42:11,932 : INFO : PROGRESS: processing document #0
2017-11-04 18:42:12,041 : INFO : PROGRESS: processing document #10000
2017-11-04 18:42:12,109 : INFO : PROGRESS: processing document #20000
2017-11-04 18:42:12,179 : INFO : PROGRESS: processing document #30000
2017-11-04 18:42:12,249 : INFO : PROGRESS: processing document #40000
2017-11-04 18:42:12,316 : INFO : PROGRESS: processing document #50000
2017-11-04 18:42:12,363 : INFO : calculating IDF weights for 56379 documents and 76640 features (1388440 matrix non-zeros)
2017-11-04 18:42:12,424 : INFO : starting similarity index under index/
2017-11-04 18:42:16,153 : INFO : PROGRESS: fresh_shard size=10000
2017-11-04 18:42:19,429 : INFO : PROGRESS: fresh_shard size=20000
2017-11-04 18:42:23,654 : INFO : PROGRESS: fresh_shard size=30000
2017-11-04 18:42:24,704 : INFO : creating sparse index
2017-11-04 18:42:24,704 : INFO : creating sparse matrix from corpus
201

In [24]:
# looks like it works
sims = index[tfidf[corpus][10]] # perform a similarity query against the corpus
sims = sorted(sims, key=lambda item: -item[1])

# [df["text"].tolist()[i[0]] for i in sims]

In [25]:
from mathviz_hopper.src.table import Table 
t = Table(index)
t.print_html()

ImportError: No module named mathviz_hopper.src.table

In [ ]:
import requests
r = requests.post('http://localhost:8082', json={"query_vec": [(0, 1), (1, 1)]})